# Data Processing

This notebook will be used to clean and process the data for HDB Resale Prices for Q1 and Q2 of the assignment.

# Imports

In [62]:
import pandas as pd
import numpy as np
import requests
import random

from math import radians, degrees, sin, cos, asin, acos, sqrt

# Functions

In [63]:
# Create Columns
def create_column(df):
    df['longtitude'] = '' # Create longtitude column
    df['latitude'] = '' # Create latitude column
    return df

In [64]:
# Function to convert block and street_name into latitude and longitude
def converter(df, name):
    row = 0

    # Loop for each block & Street Name = Address
    for block, street in zip(df['block'], df['street_name']):
        
        if (df['longtitude'][row] != ''): # If row is not empty, do nothing
            print('Row ' + str(row) + " converted")
            row = row + 1
        
        else:
            # Url of OneMap API
            url = 'https://developers.onemap.sg/commonapi/search?searchVal={0}%20{1}&returnGeom=Y&getAddrDetails=Y'.format(block, street)
            # Request API
            res = requests.get(url)

            # Check Status Code
            if res.status_code != 200:
                print("Error",res.status_code)
                break

            # Extract Geo Data
            geo_data = res.json() #Save JSON into dict

            try:
                longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
                latitude = geo_data['results'][0]['LATITUDE'] # Latitude
            except:
                longitude = 'NA'
                latitude = 'NA'
                print("No Data")

            # Add value into Longitude and Latitude columns in dataframe
            df['longtitude'][row] = longitude
            df['latitude'][row] = latitude
            
            print('Converting Row: ' + str(row))
            row = row + 1
            
    # Save DataFrame to CSV
    df.to_csv(f'../datasets/resale-flat-prices/{name}.csv', index = False)
    
    return df

In [65]:
# Function to get Mrt Station Latitude and longitutre
def mrt_station(df):
    df['mrt_lat'] = ''
    df['mrt_long'] = ''
    for town in df['town'].unique():
        url = f'https://developers.onemap.sg/commonapi/search?searchVal={town}%20MRT%20Station&returnGeom=Y&getAddrDetails=Y'
        # Request API
        res = requests.get(url)

        # Check Status Code
        if res.status_code != 200:
            print("Error",res.status_code)
            break

        # Extract Geo Data
        geo_data = res.json() #Save JSON into dict

        try:
            longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
            latitude = geo_data['results'][0]['LATITUDE'] # Latitude
        except:
            if town == 'BUKIT MERAH':
                url = 'https://developers.onemap.sg/commonapi/search?searchVal=TIONG%20BAHRU%20MRT%20Station&returnGeom=Y&getAddrDetails=Y'
                res = requests.get(url)
                geo_data = res.json() #Save JSON into dict
                longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
                latitude = geo_data['results'][0]['LATITUDE'] # Latitude
            elif town == 'CENTRAL AREA':
                url = 'https://developers.onemap.sg/commonapi/search?searchVal=RAFFLES%20PLACE%20MRT%20Station&returnGeom=Y&getAddrDetails=Y'
                res = requests.get(url)
                geo_data = res.json() #Save JSON into dict
                longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
                latitude = geo_data['results'][0]['LATITUDE'] # Latitude
            elif town == 'KALLANG/WHAMPOA':
                url = 'https://developers.onemap.sg/commonapi/search?searchVal=BOON%20KENG%20MRT%20Station&returnGeom=Y&getAddrDetails=Y'
                res = requests.get(url)
                geo_data = res.json() #Save JSON into dict
                longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
                latitude = geo_data['results'][0]['LATITUDE'] # Latitude
            else:
                url = 'https://developers.onemap.sg/commonapi/search?searchVal=YEW%20TEE%20MRT%20Station&returnGeom=Y&getAddrDetails=Y'
                res = requests.get(url)
                geo_data = res.json() #Save JSON into dict
                longitude = geo_data['results'][0]['LONGTITUDE'] # Longitude
                latitude = geo_data['results'][0]['LATITUDE'] # Latitude
                
        # Add value into Longitude and Latitude columns in dataframe
        df.loc[(df['town'] == town), 'mrt_lat'] = latitude
        df.loc[(df['town'] == town), 'mrt_long'] = longitude
    return df


In [66]:
#This is the function to calculate distance between Geo coordinates
def get_distance_km(lon1, lat1, lon2, lat2):
    lon1 = lon1.map(radians)
    lon2 = lon2.map(radians)
    lat1 = lat1.map(radians)
    lat2 = lat2.map(radians)
    
    return 6371 * ((lat1.map(sin) * lat2.map(sin) + lat1.map(cos) * lat2.map(cos) * (lon1-lon2).map(cos)).map(acos))


In [67]:
# Calculate distance to specific place
def get_distance_km_specific(lon1, lat1, lon2, lat2):
    lon1 = lon1.map(radians)
    lat1 = lat1.map(radians)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    
    return 6371 * ((lat1.map(sin) * sin(lat2) + lat1.map(cos) * cos(lat2) * (lon1-lon2).map(cos)).map(acos))

In [68]:
# Identify if the town is a mature estate. 1 = mature estate, 0 = non mature estate
def mature(df, estates):
    df['mature'] = 0
    for town in estates:
        df.loc[(df['town'] == town), 'mature'] = 1
    return df

# Import Dataset

In [69]:
# Importing Housing Dataset
df_17 = pd.read_csv("../datasets/resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
df_15_16 = pd.read_csv("../datasets/resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")
df_12_14 = pd.read_csv("../datasets/resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
df_00_12 = pd.read_csv("../datasets/resale-flat-prices/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")
df_90_99 = pd.read_csv("../datasets/resale-flat-prices/resale-flat-prices-based-on-approval-date-1990-1999.csv")

In [70]:
df_17.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


# Pre-processing

## Convert Address into Longtitude & Latitude

We are converting the block and street_name columns into longtitude and latitude so that we will be able to find out the exact location of the houses.

We can also use this data for future feature engineering, such as finding the distance to MRT stations of their respective towns and distance to CBD area (Raffles Place) or maybe even the Airport.

**Warning! Do not run the 2 cells below, it takes forever to run**

# Feature Engineering

In [71]:
# Importing Housing Dataset
df_17 = pd.read_csv("../datasets/resale-flat-prices/resale_prices_17.csv")
df_15_16 = pd.read_csv("../datasets/resale-flat-prices/resale_prices_15_16.csv")
df_12_14 = pd.read_csv("../datasets/resale-flat-prices/resale_prices_12_14.csv")
df_00_12 = pd.read_csv("../datasets/resale-flat-prices/resale_prices_00_12.csv")
df_90_99 = pd.read_csv("../datasets/resale-flat-prices/resale_prices_90_99.csv")

In [72]:
# Combine all datasets into one
df = df_90_99.append(df_00_12)
df = df.append(df_12_14)
df = df.append(df_15_16)
df = df.append(df_17)

## Create Columns

Remaining Lease

Distance from MRT

Distance from CBD

Distance from Airport

Matured Estate or not

### Calculate Remaining Lease

In [73]:
# Fill null values from remaining_lease column 
# minus current year with lease_commence_date
df['remaining_lease'] = df.apply(lambda row: 99 - (2020 - df['lease_commence_date']))

### Create latitude and longitute for respective mrt stations

In [74]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG', 'CLEMENTI',
       'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'QUEENSTOWN', 'SENGKANG',
       'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS', 'YISHUN',
       'LIM CHU KANG', 'SEMBAWANG', 'BUKIT PANJANG', 'PASIR RIS',
       'PUNGGOL'], dtype=object)

Nearest MRT: 

1) Bukit Merah = Tiong Bahru

2) Central Area = Raffles Place

3) Kallang/Whampoa = Boon Keng

4) Lim Chu Kang = YewTee

In [75]:
# Creating mrt columns
df = mrt_station(df)

### Create Distance to Mrt column

In [76]:
# Checking Datatype of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808472 entries, 0 to 62264
Data columns (total 15 columns):
block                  808472 non-null object
flat_model             808472 non-null object
flat_type              808472 non-null object
floor_area_sqm         808472 non-null float64
latitude               796252 non-null float64
lease_commence_date    808472 non-null int64
longtitude             796253 non-null float64
month                  808472 non-null object
remaining_lease        808472 non-null int64
resale_price           808472 non-null float64
storey_range           808472 non-null object
street_name            808472 non-null object
town                   808472 non-null object
mrt_lat                808472 non-null object
mrt_long               808472 non-null object
dtypes: float64(4), int64(2), object(9)
memory usage: 98.7+ MB


In [77]:
# Convert data type for mrt_lat and mrt_long
df = df.astype({'mrt_lat': 'float', 'mrt_long': 'float', 'latitude': 'float', 'longtitude': 'float'})

In [78]:
# Get distance to mrt
df['dist_mrt'] = 0
df['dist_mrt'] = get_distance_km(lon1 = df['longtitude'], lat1 = df['latitude'], lon2 = df['mrt_long'], lat2 = df['mrt_lat'])

### Create Distance to CBD (Raffles Place)
Raffles Place Mrt: lat = 1.2839332623453799, long = 103.851463066212

In [79]:
# Get Distance to CBD
df['dist_cbd'] = 0
df['dist_cbd'] = get_distance_km_specific(lon1 = df['longtitude'], lat1 = df['latitude'], lon2 = 103.851463066212, lat2 = 1.2839332623453799)

### Create Distance to Airport
Changi Airport: lat = 1.35747897447696, long = 103.98788356959

In [80]:
# Get Distance to Airport
df['dist_ap'] = 0
df['dist_ap'] = get_distance_km_specific(lon1 = df['longtitude'], lat1 = df['latitude'], lon2 = 103.98788356959, lat2 = 1.35747897447696)

### Create whether it is a mature estate or not
Based on [link](https://blog.carousell.com/property/mature-non-mature-estates-hdb-singapore/), the mature estates are: 

Ang Mo Kio, Bedok, Bishan, Bukit Merah, Bukit Timah, Central, Clementi, Geylang, Kallang/Whampoa, Marine Parade, Pasir Ris, Queenstown, Serangoon, Tampines, Toa Payoh


In [81]:
# List of Matured Estates
estates = ['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT MERAH', 'BUKIT TIMAH', 'CENTRAL', 'CLEMENTI', 'GEYLANG', 'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'QUEENSTOWN', 'SERANGOON', 'TAMPINES', 'TOA PAYOH']

In [82]:
# Create Mature Estate Column
df = mature(df, estates)

# Clean Data

## Remove Houses that are not HDB

I am dropping all the observations that are not resale HDB flats as the purpose of this project is to analyze the data and create models based on HDB buildings. So, privated houses would mess up the model and skew our data.

In [99]:
df['flat_model'].unique()

array(['NEW GENERATION', 'IMPROVED', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', '2-ROOM',
       'IMPROVED-MAISONETTE', 'MULTI GENERATION', 'PREMIUM APARTMENT',
       'ADJOINED FLAT', 'PREMIUM MAISONETTE', 'MODEL A2', 'DBSS',
       'TYPE S1', 'TYPE S2', 'PREMIUM APARTMENT LOFT'], dtype=object)

In [84]:
# Drop all the observations with Terrace flat model
df = df[(df['flat_model'] != 'Terrace') & (df['flat_model'] != 'TERRACE')]

In [85]:
# Standardize Naming or flat_model
df['flat_model'] = df['flat_model'].map(lambda x: str(x).upper())

## Null Values

In [87]:
# Explore the datatypes of features and their null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 807838 entries, 0 to 62264
Data columns (total 19 columns):
block                  807838 non-null object
flat_model             807838 non-null object
flat_type              807838 non-null object
floor_area_sqm         807838 non-null float64
latitude               795618 non-null float64
lease_commence_date    807838 non-null int64
longtitude             795619 non-null float64
month                  807838 non-null object
remaining_lease        807838 non-null int64
resale_price           807838 non-null float64
storey_range           807838 non-null object
street_name            807838 non-null object
town                   807838 non-null object
mrt_lat                807838 non-null float64
mrt_long               807838 non-null float64
dist_mrt               795618 non-null float64
dist_cbd               795618 non-null float64
dist_ap                795618 non-null float64
mature                 807838 non-null int64
dtypes: fl

The Datatype of the features are as they should be, no changes needed.

There are a few thousand rows of null values for the coordinates. This might be due to the geolocations of the address is unavailable. We will explore further before deciding whether to drop the rows.

Since the number of null values is only around 2%, it might not affect the overall result.

In [88]:
# Check which year the addresses with null values is first built
df[df['latitude'].isnull()]['lease_commence_date'].value_counts()

1978    2002
1969    1698
1974    1533
1977    1392
1973     798
1976     729
1984     707
1980     539
1981     398
1985     383
1972     334
1975     328
1971     298
1979     207
1986     201
1982     193
1970     186
1983     120
1996      91
1967      83
Name: lease_commence_date, dtype: int64

As we can see here, most building are built very long ago and very old. Hence there is a possibility that this houses might not exist anymore because of en bloc-ing. (building bought over and demolished to be rebuilt to something else)

Hence, we shall drop all the observations with null values.

In [89]:
# Drop all null observations
df = df.dropna()

## Summary Statistics

In [90]:
# Check Statistics of Dataset (It only shows numerical Values)
df.describe()

,floor_area_sqm,latitude,lease_commence_date,longtitude,remaining_lease,resale_price,mrt_lat,mrt_long,dist_mrt,dist_cbd,dist_ap,mature
count,795618.000000,795618.000000,795618.000000,795618.000000,795618.000000,7.956180e+05,795618.000000,795618.000000,795618.000000,795618.000000,795618.000000,795618.00000
mean,95.957137,1.361482,1987.126391,103.839135,66.126391,2.923675e+05,1.359827,103.837628,1.216366,12.088879,17.156694,0.51186
std,25.904908,0.041418,9.171724,0.074182,9.171724,1.463278e+05,0.041605,0.074714,0.788059,4.394001,8.275050,0.49986
min,31.000000,1.270380,1966.000000,103.685228,45.000000,5.000000e+03,1.283933,103.697322,0.039118,0.587376,2.668990,0.00000
25%,73.000000,1.334061,1980.000000,103.772776,59.000000,1.820000e+05,1.333279,103.765774,0.694575,9.164215,10.506930,0.00000
50%,93.000000,1.354403,1986.000000,103.842575,65.000000,2.745000e+05,1.353398,103.846517,1.061158,12.728522,16.721602,1.00000
75%,115.000000,1.380655,1994.000000,103.898340,73.000000,3.800000e+05,1.379561,103.895637,1.550309,15.478102,24.677444,1.00000
max,243.000000,1.457071,2016.000000,103.987805,95.000000,1.205000e+06,1.449172,103.949333,11.911097,22.393280,33.696856,1.00000


- Generally the Mean is higher than the Median in all the features. This might mean the data might be skewed slightly to the left.
- The Max value for dist_mrt and floor_area_sqm is significantly higher than the 75%. This means there might be presence of outliers

In [91]:
# Check Floor are sqm for outliers
df[df['floor_area_sqm'] > 200].head()

,block,flat_model,flat_type,floor_area_sqm,latitude,lease_commence_date,longtitude,month,remaining_lease,resale_price,storey_range,street_name,town,mrt_lat,mrt_long,dist_mrt,dist_cbd,dist_ap,mature
80647,455,MAISONETTE,EXECUTIVE,243.0,1.358728,1990,103.830643,1994-11,69,470000.0,10 TO 12,SIN MING AVE,BISHAN,1.351019,103.850057,2.322209,8.632889,17.480021,1
82586,441,MAISONETTE,EXECUTIVE,243.0,1.359084,1990,103.831751,1994-12,69,496000.0,10 TO 12,SIN MING AVE,BISHAN,1.351019,103.850057,2.223842,8.638951,17.357187,1
93494,445,MAISONETTE,EXECUTIVE,243.0,1.358711,1990,103.831720,1995-06,69,550000.0,10 TO 12,SIN MING AVE,BISHAN,1.351019,103.850057,2.210612,8.599725,17.360245,1
95453,443,MAISONETTE,EXECUTIVE,243.0,1.358948,1990,103.832788,1995-07,69,506000.0,07 TO 09,SIN MING AVE,BISHAN,1.351019,103.850057,2.112547,8.595770,17.241807,1
151792,441,MAISONETTE,EXECUTIVE,243.0,1.359084,1990,103.831751,1997-03,69,708000.0,10 TO 12,SIN MING AVE,BISHAN,1.351019,103.850057,2.223842,8.638951,17.357187,1


Maisonette is acceptable as they are still within the HDB category. No need to adjust the data.

In [92]:
# Check MRT Distance for outliers
df[df['dist_mrt'] > 10]['block'].unique()

array(['8', '6', '10'], dtype=object)

We can see that most of the huge dist_mrt values are cause by an error when retrieving the geo location from address as the are more than 1 location with similar name. I will have to fix this.

In [93]:
# Fix Wrong Geolocation for specific address
for i in ['6', '8', '10']:
    url = f'https://developers.onemap.sg/commonapi/search?searchVal={i}%20JLN%20BATU&returnGeom=Y&getAddrDetails=Y'
    res = requests.get(url)
    geo_data = res.json() #Save JSON into dict
    longitude = geo_data['results'][1]['LONGTITUDE'] # Longitude
    latitude = geo_data['results'][1]['LATITUDE'] # Latitude
    df.loc[(df['street_name']== 'JLN BATU') & (df['block'] == i), 'latitude'] = latitude
    df.loc[(df['street_name']== 'JLN BATU') & (df['block'] == i), 'longtitude'] = longitude

In [94]:
# Convert mrt_lat, mrt_long datatype
df = df.astype({'latitude': 'float', 'longtitude': 'float'})

In [95]:
# Recalculate Distance to MRT
df['dist_mrt'] = get_distance_km(lon1 = df['longtitude'], lat1 = df['latitude'], lon2 = df['mrt_long'], lat2 = df['mrt_lat'])

In [96]:
# Check Statistic to Confirm 
df.describe()

,floor_area_sqm,latitude,lease_commence_date,longtitude,remaining_lease,resale_price,mrt_lat,mrt_long,dist_mrt,dist_cbd,dist_ap,mature
count,795618.000000,795618.000000,795618.000000,795618.000000,795618.000000,7.956180e+05,795618.000000,795618.000000,795618.000000,795618.000000,795618.000000,795618.00000
mean,95.957137,1.361482,1987.126391,103.839135,66.126391,2.923675e+05,1.359827,103.837628,1.216366,12.088879,17.156694,0.51186
std,25.904908,0.041418,9.171724,0.074182,9.171724,1.463278e+05,0.041605,0.074714,0.788059,4.394001,8.275050,0.49986
min,31.000000,1.270380,1966.000000,103.685228,45.000000,5.000000e+03,1.283933,103.697322,0.039118,0.587376,2.668990,0.00000
25%,73.000000,1.334061,1980.000000,103.772776,59.000000,1.820000e+05,1.333279,103.765774,0.694575,9.164215,10.506930,0.00000
50%,93.000000,1.354403,1986.000000,103.842575,65.000000,2.745000e+05,1.353398,103.846517,1.061158,12.728522,16.721602,1.00000
75%,115.000000,1.380655,1994.000000,103.898340,73.000000,3.800000e+05,1.379561,103.895637,1.550309,15.478102,24.677444,1.00000
max,243.000000,1.457071,2016.000000,103.987805,95.000000,1.205000e+06,1.449172,103.949333,11.911096,22.393280,33.696856,1.00000


In [97]:
# Set Month as index
df['month']= pd.to_datetime(df['month'])
df.set_index('month', drop= True, inplace= True)

# Save CSV of cleaned Data

In [98]:
# Save Dataframe to csv
df.to_csv('../datasets/resale-flat-prices/clean_resale_price.csv', index = True)

Q1) Geo_Spatial Data Analysis will be done in geo_spatial.ipynb

Q2) Hedonic Regression Modelling will be done in hdb_price_prediction.ipynb